In [13]:
import pandas as pd
import os

# Set folder path
ordner_pfad = "C:/Users/anasl/Desktop/Daten 1/Daten/ods Files"

In [14]:
# Get all .ods files
ods_dateien = [datei for datei in os.listdir(ordner_pfad) if datei.endswith('.ods')]
alle_daten = []

In [15]:
# Read each .ods file and add Standort column
for datei_name in ods_dateien:
    standort_name = datei_name.replace('.ods', '')
    vollstaendiger_pfad = os.path.join(ordner_pfad, datei_name)
    daten = pd.read_excel(vollstaendiger_pfad, engine='odf')
    daten.insert(0, 'Standort', standort_name)
    alle_daten.append(daten)

# Combine all data
kombinierte_daten = pd.concat(alle_daten, ignore_index=True)

In [16]:
# Rename columns
spalten_mapping = {
    'Minimale Temperatur': 'MinTemp',
    'Mittlere Temperatur': 'AvgTemp', 
    'Maximale Temperatur': 'MaxTemp',
    'Niederschlag': 'Regen',
    'Bodentemperatur': 'Boden'
}
bereinigte_daten = kombinierte_daten.rename(columns=spalten_mapping)

# Convert Datum to datetime and filter from 2020-01-01
bereinigte_daten['Datum'] = pd.to_datetime(bereinigte_daten['Datum'], errors='coerce')
bereinigte_daten = bereinigte_daten[bereinigte_daten['Datum'] >= '2020-01-01']

In [17]:
# Drop unwanted columns
bereinigte_daten = bereinigte_daten.drop(columns=['Mittel_Luftdruck'], errors='ignore')

# Add missing columns with 0 values
bereinigte_daten['Sonne'] = 0
bereinigte_daten['Mittel_Windstaerke'] = 0
bereinigte_daten['Max_Windgeschwindigkeit'] = 0

# Reorder columns
gewuenschte_reihenfolge = [
    'Standort', 'Datum', 'MinTemp', 'AvgTemp', 'MaxTemp', 
    'Regen', 'Sonne', 'Mittel_Windstaerke', 'Max_Windgeschwindigkeit', 'Boden'
]
bereinigte_daten = bereinigte_daten[gewuenschte_reihenfolge]

In [18]:
# Fill missing values for temperature columns with mean per Standort
temp_spalten = ['Boden', 'MinTemp', 'AvgTemp', 'MaxTemp']
for spalte in temp_spalten:
    bereinigte_daten[spalte] = pd.to_numeric(bereinigte_daten[spalte], errors='coerce')
    bereinigte_daten[spalte] = bereinigte_daten.groupby('Standort')[spalte].transform(
        lambda x: x.fillna(x.mean())
    )

# Fill Sonne missing values with 0
bereinigte_daten['Sonne'] = bereinigte_daten['Sonne'].fillna(0)

In [ ]:
# Export to CSV
bereinigte_daten.to_csv("ods_combined.csv", sep=",", index=False)